# Visualisation of LFP & Sleep scoring

### Load LFP and packages

Load packages

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from scipy import signal
import os
from IPython.display import display
from ipyfilechooser import FileChooser
from ephyviewer import mkQApp, MainViewer, TraceViewer
import ephyviewer
from scipy.stats import zscore
from itertools import groupby
from ephyviewer import mkQApp, MainViewer, TraceViewer, TimeFreqViewer
from ephyviewer import InMemoryAnalogSignalSource
from ephyviewer import mkQApp, MainViewer, TraceViewer, CsvEpochSource, EpochEncoder
import json
import IPython
from matplotlib import cm
from matplotlib.colors import to_hex

%matplotlib widget

In [ ]:
#Load LFP coordinates 
notebook_path = Path("/".join(IPython.extract_module_locals()[1]["__vsc_ipynb_file__"].split("/")[-5:]))
Channels = f'{notebook_path.parent}/_LFP_coordinates_of_all_mice.csv'
all_LFPcoordinates = pd.read_csv(Channels, index_col=0)

Choose OpenEphys folder

In [ ]:
try: # tries to retrieve dpath either from a previous run or from a previous notebook
    %store -r dpath
except:
    print("the path was not defined in store")
    dpath = "//10.69.168.1/crnldata/forgetting/"

fc1 = FileChooser(dpath,select_default=True, show_only_dirs = True, title = "<b>Go inside the folder containing the LFP raw file</b>")
display(fc1)

# Sample callback function
def update_my_folder(chooser):
    global dpath
    dpath = chooser.selected
    %store dpath
    return 

# Register callback function
fc1.register_callback(update_my_folder)

Load LFPs data 

In [ ]:
folder_base = Path(dpath) 
LFPfile = Path(f'{folder_base}\continuousDS.npy')
LFPfile2 = Path(f'{folder_base}\RawDataChannelExtractedDS.npy')
LFPfile3=Path(f'{folder_base}\continuous.dat')

if LFPfile.exists(): # prefer loading downsample file over original file
    print('continuousDS.npy file')
    All = np.load(LFPfile, mmap_mode= 'r')
    samplerate=1000 
    numchannel=All.shape[1]
    filepath = Path(os.path.join(folder_base.parent.parent, f'structure.oebin'))
    if filepath.exists():    
        with open(filepath) as f:
            metadata = json.load(f)
        rec_ch_list=np.array([int(''.join(c for c in metadata['continuous'][0]['channels'][x]['channel_name'] if c.isdigit()))-1 for x in range(0, len(metadata['continuous'][0]['channels']))] )
    else:
        samplerate=1000 
        numchannel=32
        rec_ch_list=np.arange(0,numchannel,1)
    # Load LFPs timestamps 
    for file_pathTS in folder_base.parent.parent.glob('**/continuous/*/timeStampsDS.npy'):
        print('LFPs timestamps file found')
        LFPtimestamps = np.load(file_pathTS)  
elif LFPfile2.exists():
    print('RawDataChannelExtractedDS.npy file')
    LFPfile = LFPfile2
    All = np.load(LFPfile, mmap_mode= 'r')
    samplerate=1000 
    numchannel=32
    rec_ch_list=np.arange(0,numchannel,1)
elif LFPfile3.exists():
    print('continuous.dat file')
    LFPfile = LFPfile3
    DataRec = np.fromfile(LFPfile, dtype="int16")
    filepath = Path(os.path.join(folder_base.parent.parent, f'structure.oebin'))
    if filepath.exists():    
        with open(filepath) as f:
            metadata = json.load(f)
        samplerate=metadata['continuous'][0]['sample_rate']  
        numchannel=metadata['continuous'][0]['num_channels'] 
        rec_ch_list=np.array([int(''.join(c for c in metadata['continuous'][0]['channels'][x]['channel_name'] if c.isdigit()))-1 for x in range(0, len(metadata['continuous'][0]['channels']))])
    else:
        samplerate=1000 
        numchannel=32
        rec_ch_list=np.arange(0,numchannel,1)
    All = DataRec.reshape(-1,numchannel)
    # Load LFPs timestamps 
    for file_pathTS in folder_base.parent.parent.glob('**/continuous/*/timeStamps.npy'):
        print('LFPs timestamps file found')
        LFPtimestamps = np.load(file_pathTS)  
else: 
    print('no LFPs file found')

print('sample rate =', samplerate, 'Hz')
print(numchannel, 'channels recorded')
print(round(All.shape[0]/samplerate/60), 'min of recording')

Downsample LFP data to 1kHz if needed & save it

In [ ]:
if samplerate > 1000:
    datalen = np.shape(All)[0]
    new_sampling_rate = 1000 # Hz
    Nmber_points = int(datalen * new_sampling_rate / samplerate)
    AllDS=signal.resample(All, Nmber_points, axis = 0)
    LFPtimestampsDS=LFPtimestamps[::10][:-1]
    samplerate=new_sampling_rate
    np.save(f'{LFPfile.parent}/continuousDS.npy', AllDS)
    np.save(f'{file_pathTS.parent}/timeStampsDS.npy', LFPtimestampsDS)
    All=AllDS
    LFPtimestamps=LFPtimestampsDS
# eventually delete original files to gain space

Load diffential LFP (stereotrodes)

In [ ]:
mouse=[]
pos_mice=[]
for mouse_name in all_LFPcoordinates.index:
    if mouse_name in LFPfile.__str__():
        mouse.append(mouse_name)
        pos_mice.append(LFPfile.__str__().find(mouse_name)) 
mouse = [x for _, x in sorted(zip(pos_mice, mouse))] # sort mouse in the same order as they appear in the path
if len(mouse)>1: # found multiple mouse name in the path
    ID = 1 # choose 0 to see the first mouse recorded, 1 the second, 2 the third, 3 the fourth (only 4 mice can be recorded at the same time)
    print(f"/!\ Mutliple mice recorded at the same time = {mouse}. The n°{ID+1} was choosen automatically = {mouse[ID]}.")
    mouse=mouse[ID] 
elif len(mouse)==1: # found only one mouse name in the path
    ID = 0
    mouse=mouse[ID]
    
all_LFPcoordinates= all_LFPcoordinates.astype(str)
for region in all_LFPcoordinates.loc[mouse].index:
    locals()[region]=[]
    locals()[f'{region}_ch']=[]

RecordedArea=[]
ChoosenChannels=[]
combined=[]
if mouse:
    rec_ch_list_ID=rec_ch_list-ID*32
    rec_ch_list_mouse=[value for value in rec_ch_list_ID if 0 <= value <= 31]
    for col, rec_ch in enumerate(rec_ch_list_mouse):
        for idx, LFPcoord_str in enumerate(all_LFPcoordinates.loc[mouse]):
            region=all_LFPcoordinates.loc[mouse].index[idx]
            if LFPcoord_str!='nan':
                LFPcoord = LFPcoord_str.split('_')[:2] # only take into account the 2 first of electrode of that region 
                num_ch=np.where(str(rec_ch) == np.array(LFPcoord))[0]
                if len(num_ch)>0:
                    region=all_LFPcoordinates.loc[mouse].index[idx]
                    LFP=locals()[region]
                    LFP=LFP-All[:,col+np.argmax(rec_ch_list_ID>=0)] if len(LFP)>0 else All[:,col+np.argmax(rec_ch_list_ID>=0)]
                    locals()[region]=LFP
                    locals()[f'{region}_ch'].append(rec_ch)
                    break
                continue
    
    for region in all_LFPcoordinates.loc[mouse].index:
        LFP=locals()[region]
        LFP_ch=locals()[f'{region}_ch']
        if len(LFP)>0:
            combined=zscore(LFP[:,np.newaxis]) if len(combined)==0 else np.append(combined, zscore(LFP[:,np.newaxis]), axis=1)
            RecordedArea.append(region) 
            ChoosenChannels.append(LFP_ch) 
else:
    print("/!\ No mouse name found in the path OR in the csv file '_LFP_coordinates_of_all_mice.csv'")
    mouse='' # fill mouse name
    RecordedArea=['EMG','S1','PFC','CA1'] 
    EMG=All[:,0]
    S1=All[:,0]-All[:,0]
    combined=np.stack([zscore(EMG), zscore(S1)], axis=1)

print(mouse)
print(RecordedArea)
print(ChoosenChannels) 

### Score sleep manually with ephyviewer


4 stages : Wake, NREM, REM, undefined

In [ ]:
labels = ['Wake_1', 'NREM_2', 'REM_3', 'undefined_4'] # Wake, NREM sleep, REM sleep, undefined states
epoch_dur = 5 # define epoch duration in sec
winlen= 20 # default window length in sec

SleepScoring_filename = f'{dpath}\Sleep_Scoring_{len(labels)}Stages_{epoch_dur}sEpoch.csv'
source_epoch = CsvEpochSource(SleepScoring_filename, labels)

#you must first create a main Qt application (for event loop)
app = mkQApp()

t_start = 0.

#Create the main window that can contain several viewers
win = MainViewer(debug=False, show_auto_scale=True)

#create a viewer for signal
source =InMemoryAnalogSignalSource(combined, samplerate, t_start, channel_names=RecordedArea)
view1 = TraceViewer(source=source)

view1.params['xsize']= winlen
view1.params['display_labels'] = True
view1.params['scale_mode'] = 'same_for_all'
colormap = np.insert(['#88FF88', '#8888FF', '#FF8888']* 10, 0, '#FFFFFF')
for idx, ch in enumerate(RecordedArea): 
    view1.by_channel_params[f'ch{idx}', 'color'] = colormap[idx] #FF0000 red, #00FF00 green, and #0000FF blue
view1.auto_scale()

#create a viewer for the encoder itself
view2 = EpochEncoder(source=source_epoch, name='Sleep Scoring')
view2.params['xsize']= winlen
view2.params['new_epoch_step']= epoch_dur

view2.by_label_params['label0', 'color']= '#ffcd69' #AW
view2.by_label_params['label1', 'color']= '#69cfff' #NREM
view2.by_label_params['label2', 'color']= '#ff69cd' #REM
view2.by_label_params['label3', 'color']= '#8f8f8f' #undefined
view2.controls.hide()

# FFT
view3 = TimeFreqViewer(source=source, name='FFT')

view3.params['show_axis'] = True
view3.params['timefreq', 'f_start'] = 1
view3.params['timefreq', 'f_stop'] = 50
view3.params['timefreq', 'deltafreq'] = 1 #interval in Hz
view3.params['xsize']= winlen
for idx, ch in enumerate(RecordedArea): 
    view3.by_channel_params[f'ch{idx}', 'clim'] = 1
    view3.by_channel_params[f'ch{idx}', 'visible'] = True
view3.by_channel_params['ch0', 'visible'] = False # EMG needs no FFT


#show main window and run Qapp

win.add_view(view1)
win.add_view(view3)
win.add_view(view2)
win.navigation_toolbar.spinbox_xsize.setValue(winlen)
win.show()

app.exec()

# press '1', '2', '3', '4' etc, to encode state.
# or toggle 'Time range selector' and then use 'Insert within range'

6 stages : AW, QW, NREM, IS, REM, undefined

In [ ]:
labels = ['AW_1', 'QW_2', 'NREM_3', 'IS_4',  'REM_5', 'undefined_6'] # Active wake, Quiet wake, NREM sleep, Intermediate sleep, REM sleep, undefined states
epoch_dur = 5 # define epoch duration in sec
winlen= 20 # default window length in sec

SleepScoring_filename = f'{dpath}\Sleep_Scoring_{len(labels)}Stages_{epoch_dur}sEpoch.csv'
source_epoch = CsvEpochSource(SleepScoring_filename, labels)

#you must first create a main Qt application (for event loop)
app = mkQApp()

t_start = 0.

#Create the main window that can contain several viewers
win = MainViewer(debug=False, show_auto_scale=True)

#create a viewer for signal
source =InMemoryAnalogSignalSource(combined, samplerate, t_start, channel_names=RecordedArea)
view1 = TraceViewer(source=source)

view1.params['xsize']= winlen
view1.params['display_labels'] = True
view1.params['scale_mode'] = 'same_for_all'
colormap = np.insert(['#88FF88', '#8888FF', '#FF8888']* 10, 0, '#FFFFFF')
for idx, ch in enumerate(RecordedArea): 
    view1.by_channel_params[f'ch{idx}', 'color'] = colormap[idx] #FF0000 red, #00FF00 green, and #0000FF blue
view1.auto_scale()

#create a viewer for the encoder itself
view2 = EpochEncoder(source=source_epoch, name='Sleep Scoring')

view2.params['xsize']= winlen
view2.params['new_epoch_step']= epoch_dur
view2.by_label_params['label0', 'color']= '#ffcd69' #AW
view2.by_label_params['label1', 'color']= '#69ffe6' #QW
view2.by_label_params['label2', 'color']= '#69cfff' #NREM
view2.by_label_params['label3', 'color']= '#cd69ff' #IS
view2.by_label_params['label4', 'color']= '#ff69cd' #REM
view2.by_label_params['label5', 'color']= '#8f8f8f' #undefined
view2.controls.hide()

# FFT
view3 = TimeFreqViewer(source=source, name='FFT')

view3.params['show_axis'] = True
view3.params['timefreq', 'f_start'] = 1
view3.params['timefreq', 'f_stop'] = 50
view3.params['timefreq', 'deltafreq'] = 1 #interval in Hz
view3.params['xsize']= winlen
for idx, ch in enumerate(RecordedArea): 
    view3.by_channel_params[f'ch{idx}', 'clim'] = 1
    view3.by_channel_params[f'ch{idx}', 'visible'] = True
view3.by_channel_params['ch0', 'visible'] = False # EMG needs no FFT


#show main window and run Qapp

win.add_view(view1)
win.add_view(view3)
win.add_view(view2)
win.navigation_toolbar.spinbox_xsize.setValue(winlen)
win.show()

app.exec()

# press '1', '2', '3', '4' etc, to encode state.
# or toggle 'Time range selector' and then use 'Insert within range'

### Check all channels

In [ ]:
app = mkQApp()
winlen= 20 # default window length in sec
t_start = 0.

#Create the main window that can contain several viewers
win = MainViewer(debug=False, show_auto_scale=True)
view1 = TraceViewer.from_numpy(All, samplerate, t_start, 'Signals')

cmap = cm.Spectral
values = np.linspace(0, 1, All.shape[1])
colormap = [to_hex(rgb) for rgb in cmap(values)]
for idx in range(All.shape[1]): 
    view1.by_channel_params[f'ch{idx}', 'color'] = colormap[idx]

view1.params['display_labels'] = True
view1.params['scale_mode'] = 'same_for_all'
view1.auto_scale()
view1.params['xsize']= winlen
win.add_view(view1)
win.navigation_toolbar.spinbox_xsize.setValue(winlen)

#Run
win.show()
app.exec()


Run the cell bellow if you believe omnetic headstage was reverted for this mouse during this session (if you run it twice it will go back to the initial configuration)

In [ ]:
inverted_chs=np.concatenate([range(16,32,1), range(0,16,1)], axis=0)
Allflipped=All.copy()
rec_ch_list_ID=rec_ch_list-ID*32
rec_ch_list_mouse=[value for value in rec_ch_list_ID if 0 <= value <= 31]
i=np.argmax(rec_ch_list_ID>=0)
AR=np.stack([rec_ch_list_mouse, inverted_chs[rec_ch_list_mouse], np.arange(i, i+len(rec_ch_list_mouse))], axis=1)
AR_sort=AR[AR[:, 1].argsort()]
for n,r in enumerate(AR_sort[:,2]):
    Allflipped[:,r]=All[:,n+i]
    print('column n°', r, 'becomes column n°', n+i)
All= Allflipped

# Reselect channels    
all_LFPcoordinates= all_LFPcoordinates.astype(str)
for region in all_LFPcoordinates.loc[mouse].index:
    locals()[region]=[]
    locals()[f'{region}_ch']=[]

RecordedArea=[]
ChoosenChannels=[]
combined=[]
if mouse:
    rec_ch_list_ID=rec_ch_list-ID*32
    rec_ch_list_mouse=[value for value in rec_ch_list_ID if 0 <= value <= 31]
    for col, rec_ch in enumerate(rec_ch_list_mouse):
        for idx, LFPcoord_str in enumerate(all_LFPcoordinates.loc[mouse]):
            region=all_LFPcoordinates.loc[mouse].index[idx]
            if LFPcoord_str!='nan':
                LFPcoord = LFPcoord_str.split('_')[:2] # only take into account the 2 first of electrode of that region 
                num_ch=np.where(str(rec_ch) == np.array(LFPcoord))[0]
                if len(num_ch)>0:
                    region=all_LFPcoordinates.loc[mouse].index[idx]
                    LFP=locals()[region]
                    LFP=LFP-All[:,col+np.argmax(rec_ch_list_ID>=0)] if len(LFP)>0 else All[:,col+np.argmax(rec_ch_list_ID>=0)]
                    locals()[region]=LFP
                    locals()[f'{region}_ch'].append(rec_ch)
                    break
                continue
    
    for region in all_LFPcoordinates.loc[mouse].index:
        LFP=locals()[region]
        LFP_ch=locals()[f'{region}_ch']
        if len(LFP)>0:
            combined=zscore(LFP[:,np.newaxis]) if len(combined)==0 else np.append(combined, zscore(LFP[:,np.newaxis]), axis=1)
            RecordedArea.append(region) 
            ChoosenChannels.append(LFP_ch) 
else:
    print("/!\ No mouse name found in the path OR in the csv file '_LFP_coordinates_of_all_mice.csv'")
    mouse='' # fill mouse name
    RecordedArea=['EMG','S1','PFC','CA1'] 
    EMG=All[:,0]
    S1=All[:,0]-All[:,0]
    combined=np.stack([zscore(EMG), zscore(S1)], axis=1)

print(mouse)
print(RecordedArea)
print(ChoosenChannels) 

In [ ]:
np.save(f'{LFPfile.parent}/continuousDS.npy', All) # save a reverted version that will be loaded in priority 